In [1]:
import os

from dotenv import load_dotenv

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

d:\Visual Code\RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Variables

In [2]:
load_dotenv()

True

In [3]:
file = os.getenv("FILE")
persist_directory = os.getenv("PERSIST_DIRECTORY")
token = os.getenv("TOKEN")
model_id = os.getenv("MODEL_ID")
model_name = os.getenv("MODEL_NAME")

# PDFLoader

In [4]:
loader = PyPDFLoader(file)
file = loader.load()

# Recursive TextSplit

In [5]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

In [6]:
documents = recur_split.split_documents(file)

# Embeddings (HuggingFace)

In [7]:
embedding = HuggingFaceEmbeddings(model_name=model_name)

# Vector Store

In [8]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory
)

# LLM (HuggingFace)

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=token
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=token
)

pipe = pipeline(
    model=model, 
    tokenizer=tokenizer, 
    task="text-generation", 
    temperature=0.1, 
    max_new_tokens=512, 
    do_sample=False,
    repetition_penalty=1.1,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=pipe)

d:\Visual Code\RAG\venv\Lib\site-packages\transformers\models\auto\auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.64s/it]
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


# Prompt

In [10]:
token_start, token_end = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>", "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

template = """
Você é um assistente educacional.

O usuário fornecerá os dados de uma aula em formato estruturado, incluindo:
- A disciplina;
- O conteúdo;
- O objetivo da aula.

Com base no documento ENTRECOMP (que está no contexto abaixo), identifique **três competências empreendedoras** que se relacionam diretamente com essa aula.
Retorne apenas os nomes das competências, separados por vírgula e a última competências separada por e.
Exemplo de formato: Visão, Criatividade e Valorizar ideias. \n

Contexto: {context} \n\n
Pergunta: {input}
"""

prompt = ChatPromptTemplate.from_template(token_start + template + token_end)

# Chain

In [11]:
documents_chain = create_stuff_documents_chain(
    llm, 
    prompt
)

retriver_chain = create_retrieval_chain(
    retriever=vectordb.as_retriever(search_type="mmr"), 
    combine_docs_chain=documents_chain
) 

# Question

In [12]:
question = "Aula de <Matemática>, sendo o conteúdo <Frações> e o objetivo da aula é <Uso para calcular juros>."

# Answer

In [13]:
retriver_chain.invoke({"input": question})

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'input': 'Aula de <Matemática>, sendo o conteúdo <Frações> e o objetivo da aula é <Uso para calcular juros>.',
 'context': [Document(metadata={'page_label': '1', 'page': 0, 'moddate': '2025-04-28T16:59:30-03:00', 'source': '../pdf/entrecomp.pdf', 'creationdate': '2025-04-28T16:59:30-03:00', 'total_pages': 2, 'creator': 'Microsoft® Word para Microsoft 365', 'author': 'Roselaine Monteiro Moraes', 'producer': 'Microsoft® Word para Microsoft 365'}, page_content='responsável e ética. \n \nConsciência de si e autoeficácia: acreditar em si mesmo e procurar o \ndesenvolvimento contínuo, refletir sobre os seus desejos, necessidades e \naspirações a curto, médio e longo prazo, identificar e avaliar as forças \ne as fraquezas individuais e coletivas e acreditar na capacidade para \ninfluenciar o curso dos acontecimentos, apesar da incerteza, das contrariedades e dos \npossíveis obstáculos. \n \nMotivação e perseverança: é manter o foco e não desistir! Ser determinado'),
  Document(metadata={'cre